# Plotting orbits of n objects using RK 4 solver in 2D

## Konstantinos Doran SN:22007700 Date:01/03/24

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def force(r1,r2,m1,m2):
    """Returns the gravitational force on a mass, m1, at a point r1, due to another mass, m2, at a point r2.
    
    Inputs:
    r1      position of first mass as a numpy array
    r2      position of second mass as a numpy array
    m1      mass of first object
    m2      mass of second object    
    Returns:
    f_on_m1 force on mass 1 as a numpy array
    """
    # Find r_12 and its magnitude
    r12 = r2 - r1
    f_on_m1 = G*m1*m2/(np.sqrt((np.dot(r12,r12))**3))*r12
    return f_on_m1

In [ ]:
def potential(r1,r2,m1,m2):
    """Returns the gravitational potential energy of a mass, m1, at a point r1, due to another mass, m2, at a point r2.
    
    Inputs:
    r1      positions of first mass as a 2D numpy array
    r2      positions of second mass as a 2D numpy array
    m1      mass of first object
    m2      mass of second object    
    Returns:
    pot potential energy of mass 1 as a 2D numpy array
    """
    # Find r_12 and its magnitude
    r12 = r2 - r1
    pot = -G*m1*m2/np.linalg.norm(r12, axis=1)
    return pot

First I set it up to work for 2 bodies and check if it works the same as in my previous workbook. 

In [ ]:
# Set up parameters
Ndim = 2     #2D problem for now
Nsteps = 15000
Nbodies = 2
G = 1        # Appropriate units
dt = 0.001
m1 = 0.5  # Small
m2 = 1.0     # Large
masses = np.array([m1,m2])
# Initial conditions
r1 = np.array((-2/3,0))
v1 = np.array((0,np.sqrt(2/3)))
r2 = np.array((1/3,0))
v2 = np.array((0,-np.sqrt(1/6)))
y0 = np.array((r1,r2,v1,v2))

In [ ]:
def RK4_solver(fun,y0,dt,N):
    """Solve dy/dt = fun(y,t) using fourth-order RK method.
    Inputs:
    fun  f(y,t)
    y0   Initial condition array - positions and velocities of each object in the form (r1,v1,r2,v2,....)
    dt   Step size
    N    Number of steps
    Returns: two arrays of length N+1 (x and v or equivalent)
    """
    # Storage for solution
    #State vector consisting of positions and velocities of each body at each time
    # E.g   state vector at t0 = ((x1,y1), (x2,y2), (vx1,vy1), (vx2,vy2)) or simply (r1,r2,v1,v2)
    y = np.zeros((Nsteps,2*Nbodies,Ndim))
    Energies = np.zeros((Nsteps,Nbodies))
    # Initial condition
    y[0] = y0
    #print(y[0])
    t = 0
    for i in range(N-1):
        # RK4 formulae
        #print(y[i,0])
        k1 = dt*fun(y[i],t)
        k2 = dt*fun(y[i]+0.5*k1,t+0.5*dt)
        k3 = dt*fun(y[i]+0.5*k2,t+0.5*dt)
        #print(k3, y[i])
        k4 = dt*fun(y[i]+k3,t+dt)
        y[i+1]=y[i]+(k1+2*k2+2*k3+k4)/6
        t += dt
        
    return y

In [ ]:
def RHS_Orbital_motion(y,t):
    """Implements gravitational force of n bodies RHS for ODE solver
    
    Inputs:
    y_n   state vector containing r and v for each object
    t     Time (unused)
    
    Output:
    dy_n  state vector containing gradients dr and dv
    """
    statevec = np.zeros((2*Nbodies,Ndim))
    forces = np.zeros((Nbodies,Ndim))
    energy = np.zeros((Nbodies))
    #print(statevec)
    for i in range(Nbodies-1):
        for j in range(i+1,Nbodies):
            fij = force(y[i],y[j],masses[i],masses[j])
            forces[i] += fij
            forces[j] -= fij
            
    for i in range(Nbodies):
        statevec[i] = y[Nbodies+i]
        statevec[Nbodies+i] = forces[i]/masses[i]
    #f12 = force(y[0],y[1],masses[0], masses[1])      
    #f21 = -f12  
    #accel1 = f12/masses[0]
    #accel2 = f21/masses[1]
    #statevec[2] = accel1
    #statevec[3] = accel2
    #print(statevec)
    # statevec = (v1,v2,a1,a2)
    return statevec

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
#plt.plot(t_arr, orbits[:,0])
plt.plot(orbits[:,0,0], orbits[:,0,1])
plt.plot(orbits[:,1,0], orbits[:,1,1])
plt.axis("scaled")
print(orbits[0,3,:])

In [ ]:
# Create figure with correct size
fig_parts = plt.figure(figsize=(10,10), tight_layout=True)
# Initialise number of plots
rows = 5
cols = 4
num  = rows*cols
step = int(Nsteps/num)
for i in range(num):
    ax = fig_parts.add_subplot(rows,cols,i+1)
    ax.plot(orbits[:step*i,0,0], orbits[:step*i,0,1])
    ax.plot(orbits[:step*i,1,0], orbits[:step*i,1,1])
    ax.axis('scaled')
    ax.set_ylim((-0.75,0.75))
    ax.set_xlim((-0.75,0.75))
    # Add title and labels
    ax.set_title(f"time step={i*step}")

In [ ]:
def totalenergy(orbits):
    """
    """
    TotE = np.zeros((Nsteps))
    KE = np.zeros((Nbodies,Nsteps))
    PE = np.zeros((Nbodies,Nsteps))
    for i in range(Nbodies):
        KE[i] = 0.5*masses[i]*np.linalg.norm(orbits[:,Nbodies+i], axis=1)*np.linalg.norm(orbits[:,Nbodies+i], axis=1)
        for j in range(Nbodies):
            if j !=i:
                PE[i] += potential(orbits[:,i], orbits[:,j], masses[i], masses[j])
    TotE = KE + PE
    return TotE, KE, PE

In [ ]:
Energy, KE, PE = totalenergy(orbits)
print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
plt.plot(t_arr, Energy[0]+Energy[1])

In [ ]:
def totalangmomentum(orbits):
    """"""
    angularmom = np.zeros((Nbodies,Nsteps))
    for i in range(Nbodies):
        angularmom[i] = np.cross(orbits[:,i], orbits[:,Nbodies+i])*masses[i]
    totmomentum = np.sum(angularmom,axis=0)
    return totmomentum

In [ ]:
L = totalangmomentum(orbits)
print(L)
plt.plot(t_arr, L)

In [ ]:
# Set up parameters
Ndim = 2     #2D problem for now
Nsteps = 30000
Nbodies = 3
G = 1        # Appropriate units
dt = 0.001
m1 = 1.0  
m2 = 3e-6     
m3 = 3e-8
masses = np.array([m1,m2,m3])
r12 = 1
r23 = 0.0025
v2mag = np.sqrt(G*m1*(r12-(m2*r12/(m1+m2))))/r12
v3mag = v2mag- np.sqrt((G*m2*(r23-(m3*r23/(m2+m3)))))/r23
# Initial conditions
r1 = np.array((0,0))
v1 = np.array((0,0))
r2 = np.array((r12,0))
v2 = np.array((0,v2mag))
r3 = np.array((r12+r23,0))
v3 = np.array((0,v3mag))
y0 = np.array((r1,r2,r3,v1,v2,v3))

In [ ]:
t_arr = np.linspace(0,Nsteps*dt,Nsteps)
orbits = RK4_solver(RHS_Orbital_motion,y0,dt,Nsteps)

In [ ]:
#plt.plot(orbits[:,0,0], orbits[:,0,1])
#plt.plot(orbits[:,1,0], orbits[:,1,1])
plt.plot(orbits[:,2,0]-orbits[:,1,0], orbits[:,2,1]-orbits[:,1,1])
plt.axis("scaled")

In [ ]:
fig3 = plt.figure(figsize=(12,5))
ax1 = fig3.add_subplot(1,2,1)
#ax1.plot(orbits[:,0,0], orbits[:,0,1])
#ax1.plot(orbits[:,1,0], orbits[:,1,1])
ax1.plot(orbits[:,2,0]-orbits[:,1,0], orbits[:,2,1]-orbits[:,1,1])
ax1.axis('scaled')
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_title("Position of $m_3$ relative to $m_2$")
ax2 = fig3.add_subplot(1,2,2)
ax2.plot(t_arr,orbits[:,2,0]-orbits[:,1,0], label="x")
ax2.plot(t_arr,orbits[:,2,1]-orbits[:,1,1], label="y")
ax2.set_xlabel("Time")
ax2.set_ylabel("Position")
ax2.set_title("Position of mass 2 with respect to time")
ax2.legend()
# ax3 = fig3.add_subplot(1,3,3)
# ax3.plot(t_arr,orbits[:,2,0], label="x")
# ax3.plot(t_arr,orbits[:,2,1], label="y")
# ax3.set_xlabel("time")
# ax3.set_ylabel("position")
# ax3.set_title("Position of mass 3 with respect to time")
# ax3.legend()    

In [ ]:
fig3.savefig("RK4 relative m3 orbit.pdf")

Emulates the same result as in verlet method with much faster processing time!!!!

In [ ]:
Energy, KE, PE = totalenergy(orbits)
print(Energy)
#plt.plot(t_arr, Energy[0])
#plt.plot(t_arr, Energy[1])
plt.plot(t_arr, Energy[0]+Energy[1]+Energy[2])

In [ ]:
L = totalangmomentum(orbits)
print(L)
plt.plot(t_arr, L)

In [ ]:
# Plot some kind of energy (calculate if necessary) vs t
fig2 = plt.figure(figsize=(12,20))
#fig2.tight_layout()
ax1 = fig2.add_subplot(4,2,3)
ax1.plot(t_arr, Energy[0]+Energy[1]+Energy[2])
ax1.set_xlabel("Time")
ax1.set_ylabel("E")
ax1.set_title("Total Energy over time", loc = "right")

# Plot angular momentum vs t
ax2 = fig2.add_subplot(4,2,4)
ax2.plot(t_arr,L)
ax2.set_xlabel("Time")
ax2.set_ylabel("L")
ax2.set_title("Total Angular momentum over time", loc = "right")

#plot KE vs t
ax3 = fig2.add_subplot(4,2,1)
ax3.plot(t_arr, KE[0]+KE[1]+KE[2])
ax3.set_xlabel("Time")
ax3.set_ylabel("KE")
ax3.set_title("Total Kinetic Energy over time", loc = "right")

#plot PE vs t
ax4 = fig2.add_subplot(4,2,2)
ax4.plot(t_arr, PE[0]+PE[1]+PE[2])
ax4.set_xlabel("Time")
ax4.set_ylabel("PE")
ax4.set_title("Total Potential Energy over time", loc = "right")

In [ ]:
fig3.savefig("RK4 three body energy and momentum.pdf")